In [ ]:
import torch
import json
import numpy as np
from transformers import T5EncoderModel, T5Tokenizer
import os
from pathlib import Path
import ipywidgets as widgets
#from diffusers import FluxPipeline
#from IPython.display import display, Image as IPImage
#from PIL import Image


# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Create models directory
current_dir = Path.cwd()
MODELS_DIR = current_dir.parent / "data/models"
T5_MODEL_PATH = os.path.join(MODELS_DIR, "t5-v1_1-xxl")

# os.makedirs(MODELS_DIR, exist_ok=True)
# print(f"Models directory: {os.path.abspath(MODELS_DIR)}")
# print(f"T5 path: {os.path.abspath(T5_MODEL_PATH)}")
# print(f"FLUX path: {os.path.abspath(FLUX_MODEL_PATH)}")